# Perform Cyclic Pushover Analysis of an Experimentally Tested Reinforced Concrete Column 
<br></br>
* **Description** - This project is designed to develop and validate a generalised finite model of an RC column with experimental data. 
<br></br>

## Key Learnings
1. In detail knowledge about creating the RC section using fiber section concept of OpenSeesPy and visualising the created section. 
2. How to download experimental data of columns testted under a variety of scenarios from PEER and use it in your projects. 
3. How to process the cyclic load displacement data and extract the displacement data for performing a displacement controlled simulation? 
4. In details knowledge about various stages involved in creating FE model and post processing hte load displacement result. 
<br></br>

## Problem Description and Data
  Link to experimental details : https://nisee.berkeley.edu/spd/servlet/display?format=html&id=106


In [4]:
# chek if openseespy is installed in the system

#python -m pip install openseespy
#python -m pip install opsvis
#python -m pip show matplotlib
#python -m pip show numpy
#python -m pip install plotly
#python -m pip install ipynb
#python -m pip install pandas --user


### Import Necessary Modules


In [2]:
import openseespy.opensees as os
import opsvis as osv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as pg
#from ipynb.fs.defs.disp_profile_generation import dispProfile


### Geometric and Material Details

In [15]:
# === Unit Convention
mm = 1
N = 1
Pa = 1

m = 1000 * mm

inches = 0.0254 * m
ft = 12 * inches
kip = 4.45 * 10**3 * N
ksi = 6.89 * 10**6 * Pa

kN = 1000 * N
tonf = 9.80665 * kN

MPa = 10**6 * Pa
GPa = 10**9 * Pa

nDim = 2 # number of Dimension 
nDof = 3 # number of dofs per node
nEle = 5 # designated number of elements

L = 1000*mm
B = 0.35*m
D = 0.35*m
A = L*B
E = 2.e5 * N/(mm**2)
Iz = B*(D**3)/12*(mm**4)

# === Load sign convention 
#  Vertical : +ve sign represents bottom to top 
#  Horizontal : +ve sign represents left to right 

VLoad = -600*kN 
HLoad = 1 # unit lateral reference load (also act as load factor)

# === Mesh Data Generation 
numNodes = nEle + 1 
nCoords = np.zeros([numNodes, nDim])  # 0-xcoord 1-ycoord
eleTags = []
nodeTags = []

# Nodal Data Generation 
for i in range(0,numNodes):  # range from 0 to (numNodes-1)
    nCoords[i,0]=0.0 # xcoord
    nCoords[i,1]=i*(L/nEle) # ycoord
    nodeTags.append(i+1)
    
# Element Data GTeneration 
for i in range(0,nEle):
    eleTags.append(i+1)

## Preprocessing 
### OpenSees Model Generation 
### Node, Section, Material and Element Definitions